# AutoML на текстовых данных

![NLP](./imgs/tutorial_NLP_image_1.jpg)


Чуть больше про стратегии получения представлений текстов на основе представлений слов:

![NLP2](./imgs/tutorial_NLP_image_2.jpg)

Про методы случайных алгоритмов можно подробнее прочитать в [статье](https://arxiv.org/abs/1901.10444) "No Training Required: Exploring Random Encoders for Sentence Classification".


# Импорты

In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

from lightautoml.automl.presets.text_presets import TabularNLPAutoML
from lightautoml.tasks import Task
from lightautoml.addons.interpretation import LimeTextExplainer
from lightautoml.report import ReportDecoNLP

# Выключим предупреждения от HuggingFace
import transformers
transformers.logging.set_verbosity(50)

# Чтение данных

In [2]:
%%time
df = pd.read_csv("./example_data/nlp_data.csv")

CPU times: user 183 ms, sys: 39.5 ms, total: 222 ms
Wall time: 221 ms


In [3]:
print(df.shape)
df.sample(5, random_state=0)

(13842, 6)


,BankName,Message,ViewsNum,IsGood,MessageRecognized,WER
11474,Альфа-Банк,"Я клиент банка с 2007 года, зарплатный клиент ...",1422,False,Я клиент банка с две тысячи седьмого года зарп...,60.000000
3955,Альфа-Банк,07.04 в 20-15 по Ульяновскому времени я зашла ...,2016,False,Седьмого апреля в двадцать пятнадцать По Ульян...,31.818182
3081,Банк Открытие,Ужасный сервис. Заказал кредитную карту по акц...,2232,False,Ужасной Сервис заказал кредитную карту по акци...,68.750000
12107,Почта Банк,"Добрый вечер. 21.01.2020, я обратилась в отде...",1139,False,Добрый вечер двадцать первого января две тысяч...,55.555556
10494,Русский Стандарт,Банк второй месяц подряд еженедельно названива...,1609,False,Второй месяц подряд еженедельно и предлагает к...,39.393939


# Разбиение на обучающую и контрольные выборки

In [4]:
train, test = train_test_split(df, test_size=3_000, random_state=42, stratify=df.IsGood)

# Скачиваем эмбеддинги для русского языка

In [5]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar

--2021-06-14 23:58:12--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar.3’

navec_hudlit_v1_12B 100%[===================>]  50.56M  11.4MB/s    in 4.5s    

2021-06-14 23:58:16 (11.3 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar.3’ saved [53012480/53012480]



In [6]:
from navec import Navec
path = 'navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

# Обучение AutoML или День Сурка

## День 1. Стандартные параметры, ЦПУ

In [ ]:
roles = {'target': 'IsGood',
         'text': ['BankName', 'Message'],
         'drop': ['MessageRecognized', 'WER']}

task = Task('binary')

automl = TabularNLPAutoML(task = task, 
                          timeout = 3600,
                          gpu_ids = None,
                          text_params = {'lang': 'ru'},
                          verbose=2)

oof_pred = automl.fit_predict(train, roles=roles) 
not_nan = np.any(~np.isnan(oof_pred.data), axis=1)

In [8]:
print('AUC OOF score: {}'.format(roc_auc_score(train[roles['target']].values[not_nan], oof_pred.data[not_nan][:, 0])))

AUC OOF score: 0.8326628448807263


In [9]:
%%time 

test_pred = automl.predict(test)
print('AUC TEST score: {}'.format(roc_auc_score(test[roles['target']].values, test_pred.data[:, 0])))

Feature concated__BankName__Message transformed
AUC TEST score: 0.8397933778340239
CPU times: user 7.46 s, sys: 1.74 s, total: 9.2 s
Wall time: 11.5 s


## День 2. Пользовательские представления слов, ЦПУ

In [ ]:
roles = {'target': 'IsGood',
         'text': ['BankName', 'Message'],
         'drop': ['MessageRecognized', 'WER']}

task = Task('binary')

automl = TabularNLPAutoML(task = task, 
                          timeout = 3600,
                          gpu_ids = None,
                          text_params = {'lang': 'ru'},
                          autonlp_params={'model_name': 'wat', 'embedding_model': navec,
                                          'transformer_params': {'model_params': {'embed_size': 300},
                                                                 'weight_type': 'idf', 'use_svd': True}},
                          verbose=2)

oof_pred = automl.fit_predict(train, roles=roles) 
not_nan = np.any(~np.isnan(oof_pred.data), axis=1)

In [11]:
print('AUC OOF score: {}'.format(roc_auc_score(train[roles['target']].values[not_nan], oof_pred.data[not_nan][:, 0])))

AUC OOF score: 0.8331853706309897


In [12]:
%%time 

test_pred = automl.predict(test)
print('AUC TEST score: {}'.format(roc_auc_score(test[roles['target']].values, test_pred.data[:, 0])))

Feature concated__BankName__Message transformed
AUC TEST score: 0.8433377172303697
CPU times: user 11.7 s, sys: 1.82 s, total: 13.5 s
Wall time: 15.4 s


## День 3. Стандартные параметры, ГПУ

In [ ]:
roles = {'target': 'IsGood',
         'text': ['BankName', 'Message'],
         'drop': ['MessageRecognized', 'WER']}

task = Task('binary')

automl = TabularNLPAutoML(task = task, 
                          timeout = 3600,
                          gpu_ids = '1',
                          text_params = {'lang': 'ru'},
                          nn_params = {'lang': 'ru'},
                          verbose=2)

oof_pred = automl.fit_predict(train, roles=roles) 
not_nan = np.any(~np.isnan(oof_pred.data), axis=1)

In [14]:
print('AUC OOF score: {}'.format(roc_auc_score(train[roles['target']].values[not_nan], oof_pred.data[not_nan][:, 0])))

AUC OOF score: 0.895916820914043


In [15]:
%%time 

test_pred = automl.predict(test)
print('AUC TEST score: {}'.format(roc_auc_score(test[roles['target']].values, test_pred.data[:, 0])))

100%|██████████| 10/10 [00:16<00:00,  1.69s/it]


Feature concated__BankName__Message transformed


test: 100%|██████████| 188/188 [00:30<00:00,  6.11it/s]


AUC TEST score: 0.9052540592405104
CPU times: user 1min 32s, sys: 33.2 s, total: 2min 6s
Wall time: 2min 26s


## День 4. Пользовательские представления слов, ГПУ, LightGBM

In [ ]:
roles = {'target': 'IsGood',
         'text': ['BankName', 'Message'],
         'drop': ['MessageRecognized', 'WER']}

task = Task('binary')

automl = TabularNLPAutoML(task = task, 
                          timeout = 3600,
                          gpu_ids = '1',
                          general_params = {'use_algos': ['lgb']},
                          text_params = {'lang': 'ru'},
                          autonlp_params={'model_name': 'random_lstm', 'embedding_model': navec},
                          verbose=2)

oof_pred = automl.fit_predict(train, roles=roles) 
not_nan = np.any(~np.isnan(oof_pred.data), axis=1)

In [17]:
print('AUC OOF score: {}'.format(roc_auc_score(train[roles['target']].values[not_nan], oof_pred.data[not_nan][:, 0])))

AUC OOF score: 0.6699591052567759


In [18]:
%%time 

test_pred = automl.predict(test)
print('AUC TEST score: {}'.format(roc_auc_score(test[roles['target']].values, test_pred.data[:, 0])))

100%|██████████| 3/3 [00:04<00:00,  1.40s/it]


Feature concated__BankName__Message transformed
AUC TEST score: 0.6823280698997218
CPU times: user 1.19 s, sys: 717 ms, total: 1.91 s
Wall time: 4.89 s


## День 5. Выбор агрегации представлений слов, ГПУ,  линейная модель и LightGBM

In [ ]:
roles = {'target': 'IsGood',
         'text': ['BankName', 'Message'],
         'drop': ['MessageRecognized', 'WER']}

task = Task('binary')

automl = TabularNLPAutoML(task = task, 
                          timeout = 3600,
                          gpu_ids = '1',
                          general_params = {'use_algos': ['linear_l2', 'lgb']},
                          text_params = {'lang': 'ru'},
                          autonlp_params={'model_name': 'pooled_bert'},
                          verbose=2)

oof_pred = automl.fit_predict(train, roles=roles) 
not_nan = np.any(~np.isnan(oof_pred.data), axis=1)

In [20]:
print('AUC OOF score: {}'.format(roc_auc_score(train[roles['target']].values[not_nan], oof_pred.data[not_nan][:, 0])))

AUC OOF score: 0.8886449778166853


In [21]:
%%time 

test_pred = automl.predict(test)
print('AUC TEST score: {}'.format(roc_auc_score(test[roles['target']].values, test_pred.data[:, 0])))

100%|██████████| 10/10 [00:16<00:00,  1.64s/it]


Feature concated__BankName__Message transformed
AUC TEST score: 0.8930039620503403
CPU times: user 13.6 s, sys: 6.03 s, total: 19.7 s
Wall time: 27.6 s


## День 6. Выбор модели Transformers, ГПУ

rubert-tiny. Подробнее в [статье](https://habr.com/ru/post/562064/).

In [ ]:
roles = {'target': 'IsGood',
         'text': ['BankName', 'Message'],
         'drop': ['MessageRecognized', 'WER']}

task = Task('binary')

automl = TabularNLPAutoML(task = task, 
                          timeout = 3600,
                          gpu_ids = '1',
                          general_params = {'use_algos': ['nn']},
                          nn_params = {'lang': 'ru', 'bert_name': "cointegrated/rubert-tiny"},
                          verbose=2)

oof_pred = automl.fit_predict(train, roles=roles) 
not_nan = np.any(~np.isnan(oof_pred.data), axis=1)

In [23]:
print('AUC OOF score: {}'.format(roc_auc_score(train[roles['target']].values[not_nan], oof_pred.data[not_nan][:, 0])))

AUC OOF score: 0.8444397429684534


In [24]:
%%time 

test_pred = automl.predict(test)
print('AUC TEST score: {}'.format(roc_auc_score(test[roles['target']].values, test_pred.data[:, 0])))

test: 100%|██████████| 188/188 [00:03<00:00, 52.69it/s]


AUC TEST score: 0.8588585048981088
CPU times: user 9.53 s, sys: 2.83 s, total: 12.4 s
Wall time: 24.8 s


# Что дальше?

## Интерпретация

### LIME

Примерный алгоритм работы:

1. Выбирается текстовая колонка (perturb_column), с помощью которой будем интерпретировать выделенное предсказание модели. При этом все остальные признаки фиксированные.
2. Создается датасет размера n_sample (по-умолчанию 5000) путем случайных удалениий токенов (группами). Датасет бинарный (есть токен / нет токена).
3. Опционально производится отбор признаков (важных токенов) с помощью LASSO (feature_selection='lasso', можно также 'none', чтобы не производить отбор). Количество признаков равно n_feautres (10 по умолчанию).
4. Обучаем на этом объясняемую модель (линейную с весами, способ подсчета весов -- косинусное расстояние по-умолчанию, также можно и свою функцию или название расстояния из sklearn.metrics.pairwise_distances). 
5. После этого веса линейной модели и являются интерпретацией.

tips: force_order отвечает за то, использовать ли признаки как мешок слов(force_order=False) или важен их порядок (force_order=True).

In [25]:
lime = LimeTextExplainer(automl, feature_selection='lasso', force_order=False)

In [26]:
instance = test.iloc[0] # объект для интерпретации
exp = lime.explain_instance(instance, labels=(0, 1), perturb_column='Message')
exp.visualize_in_notebook(label=1)

test: 100%|██████████| 313/313 [00:05<00:00, 57.07it/s]


Добрый день! Я являюсь счастливым обладателем кредитной карты данного банка. 20го числа должен был произвести пополнение карты для того, чтобы полностью воспользоваться льготным периодом кредитования.

In [27]:
instance = test.iloc[-1] # объект для интерпретации
exp = lime.explain_instance(instance, labels=(0, 1), perturb_column='Message')
exp.visualize_in_notebook(label=1)

test: 100%|██████████| 313/313 [00:05<00:00, 54.12it/s]


Я просто в шоке от этого банка!!! После ошибки сотрудников возникла спорная ситуация и я, как добропорядочный клиент обратилась в отделение и оформила претензию лично (19.09.16) и на этом сайте (21.09.16) для более оперативного реагирования. Мне обещали рассмотреть мои обращения в течении 14 рабочих дней.

## Отчет

In [ ]:
RD = ReportDecoNLP(output_path='NLP_REPORT', 
                   report_file_name='report_nlp.html')

roles = {'target': 'IsGood',
         'text': ['BankName', 'Message'],
         'drop': ['MessageRecognized', 'WER']}

task = Task('binary')

automl = TabularNLPAutoML(task = task, 
                          timeout = 3600,
                          gpu_ids = '1',
                          general_params = {'use_algos': ['linear_l2']},
                          linear_pipeline_params = {'text_features': "embed"},
                          text_params = {'lang': 'ru'},
                          autonlp_params={'model_name': 'pooled_bert',
                                          'transformer_params': {'model_params': {'pooling': 'cls'}}},
                          verbose=2)

automl_rd = RD(automl)

oof_pred = automl_rd.fit_predict(train, roles=roles) 
not_nan = np.any(~np.isnan(oof_pred.data), axis=1)

In [29]:
print('AUC OOF score: {}'.format(roc_auc_score(train[roles['target']].values[not_nan], oof_pred.data[not_nan][:, 0])))

AUC OOF score: 0.8608724490033035


In [30]:
%%time 

test_pred = automl_rd.predict(test)
print('AUC TEST score: {}'.format(roc_auc_score(test[roles['target']].values, test_pred.data[:, 0])))

100%|██████████| 3/3 [00:16<00:00,  5.45s/it]


Feature concated__BankName__Message transformed
AUC TEST score: 0.8713822510070556
CPU times: user 13.9 s, sys: 4.86 s, total: 18.7 s
Wall time: 23.2 s


Отчет лежит [здесь](./NLP_REPORT/report_nlp.html).

## Сохранение модели

In [31]:
with open('LAMA_model.pkl', 'wb') as f:
    pickle.dump(automl_rd, f)

## Больше материалов

* Github [LightAutoML](https://github.com/sberbank-ai-lab/LightAutoML) со ссылками на все материалы.
* Канал [LAMA](https://t.me/lightautoml) в Telegram.
* Примеры на kaggle с использованием текстового функционала в условии отсутсвия доступа к интернету во время инференса: [обучение](https://www.kaggle.com/simakov/lama-bert-starter) и [инференс](https://www.kaggle.com/simakov/lama-bert-inference).